## Looped over all datasets

Use the NASS case to create a loop across all json's in the folder:

1. List all .json files in your working directory.
2. Assign each a unique ID like dataset01, dataset02, etc.
3. Save this list to a reference table (CSV).
4. Loop through each file, run the flattening code, and
5. Save output to its own folder, e.g., flattened_dataset01/.

Output:
- A dataset_reference_table.csv file with all your filenames and IDs
- A folder for each dataset:
    - flattened_dataset01/
        - dataset01_main.csv
        - dataset01_authorships.csv
        ...
    - flattened_dataset02/
        - dataset02_main.csv
    ...

Everything is neatly contained and reproducible.

In [1]:
# Step 1: List and tag all JSON files
import os
import pandas as pd

# Set the folder where the JSON files live
data_dir = "."  # or specify full path if needed

# List all .json files
json_files = [f for f in os.listdir(data_dir) if f.endswith(".json")]
json_files.sort()  # ensure consistent ordering

# Create IDs like dataset01, dataset02...
dataset_ids = [f"dataset{str(i+1).zfill(2)}" for i in range(len(json_files))]

# Create reference table
reference_table = pd.DataFrame({
    "dataset_id": dataset_ids,
    "filename": json_files
})

# Save reference table
reference_table.to_csv("dataset_reference_table.csv", index=False)
print(reference_table)

   dataset_id                                           filename
0   dataset01                                          ARMS.json
1   dataset02  Current Population Survey Food Security Supple...
2   dataset03                         Farm to School Census.json
3   dataset04    Information Resources, Inc. (IRI) InfoScan.json
4   dataset05                    NASS Census of Agriculture.json
5   dataset06                                          RUCC.json
6   dataset07  Tenure, ownership, and transition of agricultu...
7   dataset08                    food access research atlas.json
8   dataset09          food acquisition and purchase survey.json
9   dataset10         household food security survey module.json
10  dataset11         local food marketing practices survey.json
11  dataset12         quarterly food at home price database.json
12  dataset13        transition of agricultural land survey.json


In [3]:
# Step 2: Loop through each JSON and apply flattening

# Define fields to exclude from flattening
exclude_fields = [
    "keywords",
    "sustainable_development_goals",
    "mesh",
    "referenced_works",
    "related_works",
    "abstract_inverted_index",
    "plain_text",
    "concepts",
    "locations"
]

# Check if a value is a nested structure
def is_nested(val):
    return isinstance(val, (list, dict))

def deep_flatten_field(data, field, id_key="id"):
    rows = []
    for rec in data:
        parent_id = rec.get(id_key)
        items = rec.get(field, [])

        if isinstance(items, dict):
            flat = flatten(items)
            flat['parent_id'] = parent_id
            rows.append(flat)
        elif isinstance(items, list):
            for item in items:
                if isinstance(item, dict):
                    flat = flatten(item)
                    flat['parent_id'] = parent_id
                    rows.append(flat)
                else:
                    rows.append({'value': item, 'parent_id': parent_id})

    return pd.DataFrame(rows) if rows else pd.DataFrame()

In [13]:
# Step 3: Process all datasets

import json
from flatten_json import flatten

import csv

for i, row in reference_table.iterrows():
    dataset_id = row["dataset_id"]
    file_name = row["filename"]
    print(f"\n Processing {file_name} → {dataset_id}")

    # Load JSON
    with open(os.path.join(data_dir, file_name), "r", encoding="utf-8") as f:
        data = json.load(f)

    # Flatten top level
    df_main = pd.DataFrame([flatten(record) for record in data])

    # Save main flat table
    output_dir = f"flattened_{dataset_id}"
    os.makedirs(output_dir, exist_ok=True)

    # ---- Clean any old CSV files ----
    for f in os.listdir(output_dir):
        if f.endswith(".csv"):
            os.remove(os.path.join(output_dir, f))
    
    main_csv_path = os.path.join(output_dir, f"{dataset_id}_main.csv")
    df_main.to_csv(main_csv_path, index=False, quoting=csv.QUOTE_ALL, escapechar='\\')

    # Validate saved main CSV
    try:
        # Suppress DtypeWarning by adding low_memory=False
        _ = pd.read_csv(main_csv_path, low_memory=False)
        print(f"Validated: {main_csv_path}")
    except Exception as e:
        print(f"Validation error in {main_csv_path}: {e}")

    # Identify nested fields
    nested_fields = set()
    for rec in data:
        for k, v in rec.items():
            if k not in exclude_fields and is_nested(v):
                nested_fields.add(k)

    # Flatten each nested field
    for field in nested_fields:
        df_nested = deep_flatten_field(data, field)
        if not df_nested.empty:
            nested_csv_path = os.path.join(output_dir, f"{dataset_id}_{field}.csv")
            df_nested.to_csv(nested_csv_path, index=False, quoting=csv.QUOTE_ALL, escapechar='\\')

            # Validate nested saved CSV
            try:
                # Suppress DtypeWarning here too
                _ = pd.read_csv(nested_csv_path, low_memory=False)
                print(f"Validated: {nested_csv_path}")
            except Exception as e:
                print(f"Validation error in {nested_csv_path}: {e}")

    print(f"Finished processing {dataset_id} — saved to {output_dir}")


 Processing ARMS.json → dataset01
Validated: flattened_dataset01\dataset01_main.csv
Validated: flattened_dataset01\dataset01_corresponding_author_ids.csv
Validated: flattened_dataset01\dataset01_grants.csv
Validated: flattened_dataset01\dataset01_apc_list.csv
Validated: flattened_dataset01\dataset01_indexed_in.csv
Validated: flattened_dataset01\dataset01_primary_topic.csv
Validated: flattened_dataset01\dataset01__id.csv
Validated: flattened_dataset01\dataset01_counts_by_year.csv
Validated: flattened_dataset01\dataset01_citation_normalized_percentile.csv
Validated: flattened_dataset01\dataset01_cited_by_percentile_year.csv
Validated: flattened_dataset01\dataset01_primary_location.csv
Validated: flattened_dataset01\dataset01_apc_paid.csv
Validated: flattened_dataset01\dataset01_topics.csv
Validated: flattened_dataset01\dataset01_dataset.csv
Validated: flattened_dataset01\dataset01_best_oa_location.csv
Validated: flattened_dataset01\dataset01_ids.csv
Validated: flattened_dataset01\datase

## Generate a Schema Diagram

In [3]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Graphviz\bin"

import shutil
print(shutil.which("dot"))  # Should now return the full path

C:\Program Files\Graphviz\bin\dot.EXE


In [ ]:
import pandas as pd
from graphviz import Digraph

# Set directory where flattened outputs are
base_dir = "."  # or full path

# Get all dataset folders
dataset_folders = [f for f in os.listdir(base_dir) if f.startswith("flattened_dataset")]

# Loop through each dataset
for dataset_folder in dataset_folders:
    print(f"Building schema for {dataset_folder}")
    dot = Digraph(comment=f"Schema: {dataset_folder}", format="png")
    dot.attr(rankdir="LR")

    csv_files = [f for f in os.listdir(dataset_folder) if f.endswith(".csv")]

    tables = {}

    for csv_file in csv_files:
        table_name = csv_file.replace(".csv", "")
        path = os.path.join(dataset_folder, csv_file)

        # Read only first few rows for speed
        df = pd.read_csv(path, nrows=500)

        # Summarize schema
        col_lines = []
        max_cols = 10  # show up to 10 fields
        for i, col in enumerate(df.columns[:max_cols]):
            dtype = str(df[col].dtype)
            col_lines.append(f"{col} : {dtype}")
        if len(df.columns) > max_cols:
            col_lines.append("...")  # indicate there's more

        table_def = f"{{{table_name}|{'\\l'.join(col_lines)}\\l}}"

        # Store
        tables[table_name] = {
            "cols": df.columns.tolist(),
            "definition": table_def
        }
        dot.node(table_name, label=table_def, shape="record")

    # Draw relationships (parent_id → id)
    for table_name, info in tables.items():
        cols = info["cols"]
        if "parent_id" in cols:
            dot.edge(table_name, f"{dataset_folder}_main", label="parent_id → id")

    # Save diagram
    output_path = os.path.join(dataset_folder, f"{dataset_folder}_schema")
    dot.render(output_path, cleanup=True)
    print(f"Saved schema diagram to {output_path}.png")

In [16]:
import os
import pandas as pd

# Path to the dataset folder (e.g., flattened_dataset01)
dataset_folder = "flattened_dataset01"
dataset_id = "dataset01"  # used to build column links

schema_rows = []

# Loop through each CSV file
for csv_file in os.listdir(dataset_folder):
    if csv_file.endswith(".csv"):
        table_name = csv_file.replace(".csv", "")
        file_path = os.path.join(dataset_folder, csv_file)

        # Read a sample of rows
        df = pd.read_csv(file_path, nrows=100)

        for col in df.columns:
            dtype = str(df[col].dtype)

            # Determine lookup logic
            if col == "id" and "main" in table_name:
                is_lookup = "PRIMARY KEY"
            elif col == "parent_id":
                is_lookup = f"FOREIGN KEY → {dataset_id}_main.id"
            else:
                is_lookup = ""

            schema_rows.append({
                "table_name": table_name,
                "column_name": col,
                "data_type": dtype,
                "is_lookup": is_lookup
            })

# Save schema summary to CSV
schema_df = pd.DataFrame(schema_rows)
schema_df.to_csv(f"{dataset_folder}_schema_summary.csv", index=False)

print(f"Saved schema summary to {dataset_folder}_schema_summary.csv")

Saved schema summary to flattened_dataset01_schema_summary.csv
